<a href="https://colab.research.google.com/github/joshtburdick/misc/blob/master/countingBound/py/fractions/Two_phase_simplex_algo_from_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on prompt to Gemini:
Implement the two-phase simplex algorithm in Python...

Compiles and runs, but test results appear incorrect to me.

In [79]:
# prompt: add imports for using the scipy simplex solver

from scipy.optimize import linprog

import numpy as np
import random

In [80]:
from fractions import Fraction
from collections import defaultdict

def pivot(tableau, row, col):
    """Performs a pivot operation on the tableau."""
    pivot_value = tableau[row][col]
    if pivot_value == 0:
        raise ValueError("Pivot element is zero.")

    new_tableau = defaultdict(dict)

    # Update the pivot row
    new_tableau[row] = {
        c: Fraction(tableau[row].get(c, 0), pivot_value) for c in tableau[row]
    }

    # Update other rows
    for r in tableau:
        if r != row:
            factor = tableau[r].get(col, Fraction(0))
            new_tableau[r] = {}
            all_cols = set(tableau[r].keys()) | set(tableau[row].keys())
            for c in all_cols:
                term_to_subtract = factor * new_tableau[row].get(c, Fraction(0))
                new_tableau[r][c] = tableau[r].get(c, Fraction(0)) - term_to_subtract
    return new_tableau

def find_entering_variable(tableau, phase):
    """Finds the entering variable based on the objective row."""
    objective_row = tableau.get("obj", {})
    if phase == 1:
        # In Phase 1, we want to minimize the sum of artificial variables.
        # Look for a positive coefficient in the objective row (excluding the constant term).
        for col, coeff in objective_row.items():
            if col != "const" and coeff > 0:
                return col
        return None
    elif phase == 2:
        # In Phase 2, we want to maximize the original objective function.
        # Look for a negative coefficient in the objective row (excluding the constant term).
        for col, coeff in objective_row.items():
            if col != "const" and coeff < 0:
                return col
        return None
    return None

def find_leaving_variable(tableau, entering_col):
    """Finds the leaving variable using the minimum ratio test."""
    min_ratio = float('inf')
    leaving_row = None
    for row, row_data in tableau.items():
        if row != "obj":
            a_ij = row_data.get(entering_col, Fraction(0))
            if a_ij > 0:
                ratio = row_data.get("const", Fraction(0)) / a_ij
                if ratio < min_ratio:
                    min_ratio = ratio
                    leaving_row = row
    return leaving_row

def is_optimal(tableau, phase):
    """Checks if the current tableau is optimal."""
    objective_row = tableau.get("obj", {})
    if phase == 1:
        # Optimal in Phase 1 if all artificial variable coefficients in obj are non-negative (ideally zero).
        for col, coeff in objective_row.items():
            if col.startswith("a") and coeff > 0:
                return False
        return True
    elif phase == 2:
        # Optimal in Phase 2 if all non-basic variable coefficients in obj are non-positive.
        for col, coeff in objective_row.items():
            if col != "const" and coeff < 0:
                return False
        return True
    return True

def get_basic_variables(tableau):
    """Identifies the basic variables in the current tableau."""
    basic_vars = {}
    num_constraints = len(tableau) - 1 if "obj" in tableau else len(tableau)
    num_vars = 0
    if num_constraints > 0:
        first_row = next(iter(r for r in tableau if r != "obj"), {})
        num_vars = len(first_row) - 1 if "const" in first_row else len(first_row)

    for j in range(num_vars):
        col_name = f"x{j+1}"  # Assuming initial variables are x1, x2, ...
        count_nonzero = 0
        basic_row = None
        coeff_val = Fraction(0)
        for row in tableau:
            if row != "obj":
                coeff = tableau[row].get(col_name, Fraction(0))
                if coeff != 0:
                    count_nonzero += 1
                    basic_row = row
                    coeff_val = coeff
        if count_nonzero == 1 and coeff_val == 1:
            basic_vars[col_name] = tableau[basic_row].get("const", Fraction(0))
        elif count_nonzero == 1 and coeff_val != 1:
            # This shouldn't happen in a properly pivoted tableau
            print("this shouldn't happen")
            basic_vars[col_name] = tableau[basic_row].get("const", Fraction(0)) / coeff_val

    # Check for artificial variables that are basic at zero in Phase 1
    if "obj" in tableau:
        for row in list(tableau.keys()):
            if row.startswith("a"):
                row_data = tableau[row]
                count_nonzero = 0
                basic_col = None
                for col, coeff in row_data.items():
                    if col != "const" and coeff != 0:
                        count_nonzero += 1
                        basic_col = col
                if count_nonzero == 1 and row_data.get("const", Fraction(0)) == 0 and basic_col is not None and basic_col not in basic_vars:
                    basic_vars[basic_col] = Fraction(0)

    return basic_vars

def two_phase_simplex(objective, constraints, artificial_vars):
    """
    Implements the two-phase simplex algorithm for solving linear programming problems.

    Args:
        objective (dict): A dictionary representing the objective function.
                          Keys are variable names (strings like 'x1', 'x2') and 'const' for the constant term.
                          Values are the corresponding coefficients (fractions.Fraction).
                          For maximization problems, the coefficients should be negated initially.
        constraints (list of dict): A list of dictionaries, where each dictionary represents a constraint.
                                    Keys are variable names and 'const' for the right-hand side.
                                    Values are the coefficients.
                                    Each constraint dictionary should also have a 'type' key with values
                                    '<=' (less than or equal to), '>=' (greater than or equal to), or '=' (equal to).
        artificial_vars (int): The number of artificial variables introduced.

    Returns:
        tuple: A tuple containing:
            - 'optimal': If an optimal solution is found.
            - 'infeasible': If the problem is infeasible.
            - 'unbounded': If the problem is unbounded (not fully implemented for Phase 2).
            - solution (dict): A dictionary containing the values of the basic variables at the optimal solution.
                             Only present if the solution is optimal.
            - optimal_value (fractions.Fraction): The optimal value of the objective function.
                                                 Only present if the solution is optimal.
    """

    tableau_phase1 = defaultdict(dict)
    num_artificial = 0

    # Initialize Phase 1 tableau
    tableau_phase1["obj"] = defaultdict(Fraction)
    for var, coeff in objective.items():
        if var != "const":
            tableau_phase1["obj"][var] = Fraction(0)
        else:
            tableau_phase1["obj"]["const"] = Fraction(0)

    row_index = 0
    for i, constraint in enumerate(constraints):
        tableau_phase1[f"c{row_index}"] = defaultdict(Fraction)
        for var, coeff in constraint.items():
            if var != "type":
                tableau_phase1[f"c{row_index}"][var] = coeff
            tableau_phase1[f"c{row_index}"]["const"] = constraint.get("const", Fraction(0))

        if constraint["type"] == "<=":
            slack_var = f"s{i+1}"
            tableau_phase1[f"c{row_index}"][slack_var] = Fraction(1)
        elif constraint["type"] == ">=":
            slack_var = f"s{i+1}"
            artificial_var = f"a{num_artificial + 1}"
            tableau_phase1[f"c{row_index}"][slack_var] = Fraction(-1)
            tableau_phase1[f"c{row_index}"][artificial_var] = Fraction(1)
            tableau_phase1["obj"][artificial_var] = Fraction(1)
            num_artificial += 1
        elif constraint["type"] == "=":
            artificial_var = f"a{num_artificial + 1}"
            tableau_phase1[f"c{row_index}"][artificial_var] = Fraction(1)
            tableau_phase1["obj"][artificial_var] = Fraction(1)
            num_artificial += 1

        row_index += 1

    # Perform Phase 1
    print("\n--- Phase 1 ---")
    iteration = 0
    while not is_optimal(tableau_phase1, 1):
        iteration += 1
        print(f"Iteration {iteration} (Phase 1):")
        for row, data in tableau_phase1.items():
            print(f"{row}: {dict(data)}")

        entering_col = find_entering_variable(tableau_phase1, 1)
        if entering_col is None:
            break  # Should not happen if not optimal

        leaving_row = find_leaving_variable(tableau_phase1, entering_col)
        if leaving_row is None:
            print("Phase 1: Problem is unbounded (this shouldn't happen in Phase 1 for feasibility).")
            return "unbounded", None, None, None

        print(f"Entering variable: {entering_col}, Leaving variable: {leaving_row}")
        tableau_phase1 = pivot(tableau_phase1, leaving_row, entering_col)

    print("\nPhase 1 Optimal Tableau:")
    for row, data in tableau_phase1.items():
        print(f"{row}: {dict(data)}")

    # Check if Phase 1 resulted in a feasible solution for the original problem
    if tableau_phase1["obj"].get("const", Fraction(0)) > 0:
        print("\nProblem is infeasible.")
        return "infeasible", None, None, None

    # Initialize Phase 2 tableau
    tableau_phase2 = defaultdict(dict)
    tableau_phase2["obj"] = objective.copy()
    for row in [r for r in tableau_phase1 if not r.startswith("obj")]:
        tableau_phase2[row] = {k: v for k, v in tableau_phase1[row].items() if not k.startswith("a")}

    # Ensure the Phase 2 objective function is correctly expressed in terms of non-basic variables
    basic_vars_phase1 = get_basic_variables(tableau_phase1)
    obj_phase2 = tableau_phase2["obj"].copy()
    for var, val in basic_vars_phase1.items():
        if var in obj_phase2:
            coeff = obj_phase2.pop(var)
            for col, v in tableau_phase2[list(r for r in tableau_phase2 if var in tableau_phase2[r])[0]].items():
                if col != var:
                    obj_phase2[col] = obj_phase2.get(col, Fraction(0)) - coeff * v
            obj_phase2["const"] = obj_phase2.get("const", Fraction(0)) + coeff * val
    tableau_phase2["obj"] = obj_phase2

    print("\n--- Phase 2 ---")
    iteration = 0
    while not is_optimal(tableau_phase2, 2):
        iteration += 1
        print(f"Iteration {iteration} (Phase 2):")
        for row, data in tableau_phase2.items():
            print(f"{row}: {dict(data)}")

        entering_col = find_entering_variable(tableau_phase2, 2)
        if entering_col is None:
            break  # Optimal solution found

        leaving_row = find_leaving_variable(tableau_phase2, entering_col)
        if leaving_row is None:
            print("Phase 2: Problem is unbounded.")
            return "unbounded", None, None, None

        print(f"Entering variable: {entering_col}, Leaving variable: {leaving_row}")
        tableau_phase2 = pivot(tableau_phase2, leaving_row, entering_col)

    print("\nPhase 2 Optimal Tableau:")
    for row, data in tableau_phase2.items():
        print(f"{row}: {dict(data)}")

    optimal_solution = get_basic_variables(tableau_phase2)
    optimal_value = tableau_phase2["obj"].get("const", Fraction(0))

    return "optimal", optimal_solution, optimal_value

if __name__ == '__main__':
    pass


**Explanation:**

1.  **`pivot(tableau, row, col)`:**
    * Performs the pivot operation around the element at `tableau[row][col]`.
    * Divides the pivot row by the pivot element.
    * For all other rows, subtracts a multiple of the new pivot row to make the element in the pivot column zero.
    * Uses `fractions.Fraction` for exact arithmetic.
    * Represents the new tableau as a `defaultdict(dict)`.

2.  **`find_entering_variable(tableau, phase)`:**
    * **Phase 1:** Selects the entering variable

In [81]:
# this one worked (note that x1 == x2 == 0 is feasible)
if True:
    # Example 1: Maximize x1 + 2x2
    # Subject to:
    # x1 + x2 <= 4
    # 2x1 + x2 <= 5
    # x1, x2 >= 0
    objective1 = {"x1": Fraction(-1), "x2": Fraction(-2), "const": Fraction(0)} # Negated for maximization
    constraints1 = [
        {"x1": Fraction(1), "x2": Fraction(1), "type": "<=", "const": Fraction(4)},
        {"x1": Fraction(2), "x2": Fraction(1), "type": "<=", "const": Fraction(5)},
    ]
    artificial_vars1 = 0
    result1 = two_phase_simplex(objective1, constraints1, artificial_vars1)
    print("\n--- Example 1 Result ---")
    print(f"Status: {result1[0]}")
    if result1[0] == "optimal":
        print(f"Optimal Solution: {result1[1]}")
        print(f"Optimal Value: {result1[2]}")



--- Phase 1 ---

Phase 1 Optimal Tableau:
obj: {'x1': Fraction(0, 1), 'x2': Fraction(0, 1), 'const': Fraction(0, 1)}
c0: {'x1': Fraction(1, 1), 'const': Fraction(4, 1), 'x2': Fraction(1, 1), 's1': Fraction(1, 1)}
c1: {'x1': Fraction(2, 1), 'const': Fraction(5, 1), 'x2': Fraction(1, 1), 's2': Fraction(1, 1)}

--- Phase 2 ---
Iteration 1 (Phase 2):
obj: {'x1': Fraction(-1, 1), 'x2': Fraction(-2, 1), 'const': Fraction(0, 1)}
c0: {'x1': Fraction(1, 1), 'const': Fraction(4, 1), 'x2': Fraction(1, 1), 's1': Fraction(1, 1)}
c1: {'x1': Fraction(2, 1), 'const': Fraction(5, 1), 'x2': Fraction(1, 1), 's2': Fraction(1, 1)}
Entering variable: x1, Leaving variable: c1
Iteration 2 (Phase 2):
c1: {'x1': Fraction(1, 1), 'const': Fraction(5, 2), 'x2': Fraction(1, 2), 's2': Fraction(1, 2)}
obj: {'s2': Fraction(1, 2), 'x2': Fraction(-3, 2), 'x1': Fraction(0, 1), 'const': Fraction(5, 2)}
c0: {'const': Fraction(3, 2), 's1': Fraction(1, 1), 's2': Fraction(-1, 2), 'x2': Fraction(1, 2), 'x1': Fraction(0, 1)}
E

In [82]:
def test_two_phase_simplex(n, num_inequalities, num_equalities):
    """Compares the output of two_phase_simplex() to scipy's linprog.

    FIXME require this to give a bounded solution?
    Assumes the objective function is maximization, and that all
    variables being 0 is a feasible solution.
    Args:
    n: Number of variables
    num_inequalities: Number of inequality constraints
    num_equalities: Number of equality constraints
    Returns:
        bool: True if the tests pass, False otherwise.
    """
    def rand_fraction():
        return Fraction(random.randint(-100, 100), random.randint(1, 100))
    # objective function
    c = [rand_fraction()
        for i in range(n)]
    # inequality constraints
    A_ub = [[rand_fraction() for _ in range(n)] for _ in range(num_inequalities)]
    b_ub = [abs(rand_fraction()) for _ in range(num_inequalities)]
    # equality constraints
    A_eq = [[rand_fraction() for _ in range(n)] for _ in range(num_equalities)]
    b_eq = [0 for _ in range(num_equalities)]
    # this should be feasible, with all x==0
    scipy_result = linprog(c, A_ub=A_ub, b_ub=b_ub,
                           A_eq=A_eq, b_eq=b_eq,method="highs")
    return {
        "n": n,
        "num_inequalities": num_inequalities,
        "num_equalities": num_equalities,
        "scipy_result": scipy_result,
        "c1": c,
        "A_ub": A_ub,
        "b_ub": b_ub,
        "A_eq": A_eq,
        "b_eq": b_eq,
    }

In [83]:
# Example of this: note that without enough constraints, this
# may be unbounded.
r = test_two_phase_simplex(4, 7, 1)
# r["scipy_result"]["fun"]
r
# ??? why is the objective function value 0 so often?

{'n': 4,
 'num_inequalities': 7,
 'num_equalities': 1,
 'scipy_result':        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 0.0
              x: [ 0.000e+00 -0.000e+00  0.000e+00  0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00 -0.000e+00  0.000e+00  0.000e+00]
                 marginals: [ 4.445e+00  0.000e+00  8.316e+00  7.703e+00]
          upper:  residual: [       inf        inf        inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
          eqlin:  residual: [ 0.000e+00]
                 marginals: [ 8.679e+00]
        ineqlin:  residual: [ 1.284e+00  5.699e-01  3.031e+00  5.000e-01
                              3.243e-01  1.044e+00  3.763e-01]
                 marginals: [-0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00
                             -0.000e+00 -0.000e+00 -0.000e+00],
 'c1': [Fraction(-47, 48),
  Fraction(-27, 16),
  Fra

We need to convert this to the format the Gemini code expects.

In [84]:
# prompt: Write a function which, given a sample problem from test_two_phase_simplex(), runs two_phase_simplex() on it, and then compares the result with 'scipy_result', the answer from Scipy.

def run_and_compare_simplex(sample_problem):
    """
    Runs the two_phase_simplex on a sample problem and compares the result to scipy's linprog.

    Args:
        sample_problem (dict): A dictionary containing the problem definition, including
                               'scipy_result', 'c1', 'A_ub', 'b_ub', 'A_eq', 'b_eq'.

    Returns:
        tuple: A tuple containing the status of the comparison and the results.
               ('match', simplex_status, simplex_optimal_value, scipy_status, scipy_optimal_value) if they match.
               ('mismatch', simplex_status, simplex_optimal_value, scipy_status, scipy_optimal_value) if they don't.
               ('scipy_error', scipy_status, scipy_message) if scipy failed.
               ('simplex_error', status) if simplex failed in an unexpected way.
    """
    scipy_result = sample_problem["scipy_result"]
    c = sample_problem["c1"]
    A_ub = sample_problem["A_ub"]
    b_ub = sample_problem["b_ub"]
    A_eq = sample_problem["A_eq"]
    b_eq = sample_problem["b_eq"]

    # Convert scipy format to two_phase_simplex format
    objective = {"const": Fraction(0)}
    for i, coeff in enumerate(c):
        objective[f"x{i+1}"] = Fraction(-coeff)  # Negate for maximization

    constraints = []
    artificial_vars_count = 0

    for i, row in enumerate(A_ub):
        constraint = {"type": "<=", "const": Fraction(b_ub[i])}
        for j, coeff in enumerate(row):
            constraint[f"x{j+1}"] = Fraction(coeff)
        constraints.append(constraint)

    for i, row in enumerate(A_eq):
        constraint = {"type": "=", "const": Fraction(b_eq[i])}
        for j, coeff in enumerate(row):
            constraint[f"x{j+1}"] = Fraction(coeff)
        # For equality constraints in two_phase_simplex, we need an artificial variable
        # This count is used during tableau initialization.
        artificial_vars_count += 1
        constraints.append(constraint)

    # Note: The original code's artificial_vars argument is not directly used
    # in the two_phase_simplex function's logic for creating the tableau.
    # The tableau creation logic implicitly adds artificial variables based on constraints.
    # We pass the count here just to match the function signature, but it doesn't
    # control the tableau construction.

    try:
        simplex_status, simplex_solution, simplex_optimal_value = two_phase_simplex(
            objective, constraints, artificial_vars_count
        )
    except Exception as e:
        print(f"Simplex failed with error: {e}")
        return "simplex_error", str(e)

    scipy_status = scipy_result["status"]
    scipy_optimal_value = Fraction(scipy_result["fun"]) if scipy_result["fun"] is not None else None

    print("\n--- Comparison ---")
    print(f"Simplex Status: {simplex_status}")
    print(f"Scipy Status: {scipy_status}")

    # Map SciPy statuses to Simplex statuses
    scipy_status_map = {
        0: "optimal",  # Optimization terminated successfully.
        1: "infeasible",  # Problem appears to be infeasible.
        2: "unbounded",  # Problem appears to be unbounded.
        3: "stalled",  # Optimization terminated because no further progress could be made.
        4: "numerical_error",  # Numerical difficulties encountered.
    }
    mapped_scipy_status = scipy_status_map.get(scipy_status, "unknown_scipy_status")

    comparison_status = "mismatch" # Default to mismatch

    if simplex_status == "optimal" and mapped_scipy_status == "optimal":
        # Compare optimal values. Allow for floating point tolerance if needed,
        # but with Fractions, we can compare exactly.
        if simplex_optimal_value == scipy_optimal_value:
            comparison_status = "match"
        else:
            comparison_status = "mismatch" # Values don't match
            print(f"Optimal value mismatch: Simplex {simplex_optimal_value}, Scipy {scipy_optimal_value}")

    elif simplex_status == "infeasible" and mapped_scipy_status == "infeasible":
        comparison_status = "match"

    elif simplex_status == "unbounded" and mapped_scipy_status == "unbounded":
        comparison_status = "match"

    print(f"Comparison Status: {comparison_status}")

    return comparison_status, simplex_status, simplex_optimal_value, mapped_scipy_status, scipy_optimal_value

# Example usage:
# Run the test function to get a sample problem
sample_problem = test_two_phase_simplex(4, 7, 1)

# Run the comparison function
comparison_result = run_and_compare_simplex(sample_problem)
print(f"\nFinal Comparison Result: {comparison_result}")

# You can run this multiple times with different parameters for test_two_phase_simplex
# sample_problem_2 = test_two_phase_simplex(3, 5, 2)
# comparison_result_2 = run_and_compare_simplex(sample_problem_2)
# print(f"\nFinal Comparison Result 2: {comparison_result_2}")



--- Phase 1 ---
Iteration 1 (Phase 1):
obj: {'const': Fraction(0, 1), 'x1': Fraction(0, 1), 'x2': Fraction(0, 1), 'x3': Fraction(0, 1), 'x4': Fraction(0, 1), 'a1': Fraction(1, 1)}
c0: {'const': Fraction(27, 29), 'x1': Fraction(-43, 91), 'x2': Fraction(-1, 39), 'x3': Fraction(27, 31), 'x4': Fraction(1, 29), 's1': Fraction(1, 1)}
c1: {'const': Fraction(9, 11), 'x1': Fraction(50, 29), 'x2': Fraction(-22, 17), 'x3': Fraction(3, 7), 'x4': Fraction(-7, 93), 's2': Fraction(1, 1)}
c2: {'const': Fraction(20, 43), 'x1': Fraction(-30, 7), 'x2': Fraction(75, 14), 'x3': Fraction(-19, 12), 'x4': Fraction(-3, 74), 's3': Fraction(1, 1)}
c3: {'const': Fraction(74, 81), 'x1': Fraction(39, 85), 'x2': Fraction(1, 9), 'x3': Fraction(-85, 26), 'x4': Fraction(6, 71), 's4': Fraction(1, 1)}
c4: {'const': Fraction(70, 1), 'x1': Fraction(1, 73), 'x2': Fraction(30, 19), 'x3': Fraction(98, 45), 'x4': Fraction(-23, 14), 's5': Fraction(1, 1)}
c5: {'const': Fraction(23, 66), 'x1': Fraction(-1, 11), 'x2': Fraction(-1

Running this several times, it seems to mostly agree when the optimal value is 0...

The following cells are for practice:

In [85]:
# check that abs() works on fractions
abs(Fraction(-3, 1))

Fraction(3, 1)

### Original tests from Gemini


In [86]:
# this one isn't working (note that x1 == x2 == 0 isn't feasible)
if True:
    # Example 2: Minimize x1 + x2
    # Subject to:
    # x1 + x2 >= 6
    # x1 >= 2
    # x2 >= 2
    # x1, x2 >= 0
    objective2 = {"x1": Fraction(1), "x2": Fraction(1), "const": Fraction(0)}
    constraints2 = [
        {"x1": Fraction(1), "x2": Fraction(1), "type": ">=", "const": Fraction(6)},
        {"x1": Fraction(1), "type": ">=", "const": Fraction(2)},
        {"x2": Fraction(1), "type": ">=", "const": Fraction(2)},
    ]
    artificial_vars2 = 3
    result2 = two_phase_simplex(objective2, constraints2, artificial_vars2)
    print("\n--- Example 2 Result ---")
    print(f"Status: {result2[0]}")
    if result2[0] == "optimal":
        print(f"Optimal Solution: {result2[1]}")
        print(f"Optimal Value: {result2[2]}")



--- Phase 1 ---
Iteration 1 (Phase 1):
obj: {'x1': Fraction(0, 1), 'x2': Fraction(0, 1), 'const': Fraction(0, 1), 'a1': Fraction(1, 1), 'a2': Fraction(1, 1), 'a3': Fraction(1, 1)}
c0: {'x1': Fraction(1, 1), 'const': Fraction(6, 1), 'x2': Fraction(1, 1), 's1': Fraction(-1, 1), 'a1': Fraction(1, 1)}
c1: {'x1': Fraction(1, 1), 'const': Fraction(2, 1), 's2': Fraction(-1, 1), 'a2': Fraction(1, 1)}
c2: {'x2': Fraction(1, 1), 'const': Fraction(2, 1), 's3': Fraction(-1, 1), 'a3': Fraction(1, 1)}
Entering variable: a1, Leaving variable: c0
Iteration 2 (Phase 1):
c0: {'x1': Fraction(1, 1), 'const': Fraction(6, 1), 'x2': Fraction(1, 1), 's1': Fraction(-1, 1), 'a1': Fraction(1, 1)}
obj: {'a2': Fraction(1, 1), 'const': Fraction(-6, 1), 's1': Fraction(1, 1), 'a3': Fraction(1, 1), 'x2': Fraction(-1, 1), 'x1': Fraction(-1, 1), 'a1': Fraction(0, 1)}
c1: {'a2': Fraction(1, 1), 'const': Fraction(2, 1), 's1': Fraction(0, 1), 's2': Fraction(-1, 1), 'x2': Fraction(0, 1), 'x1': Fraction(1, 1), 'a1': Fractio

In [87]:
# this is indeed infeasible, but isn't getting detected as such
if True:
    # Example 3: Infeasible problem
    objective3 = {"x1": Fraction(-1), "x2": Fraction(-1), "const": Fraction(0)}
    constraints3 = [
        {"x1": Fraction(1), "x2": Fraction(1), "type": "<=", "const": Fraction(1)},
        {"x1": Fraction(1), "x2": Fraction(1), "type": ">=", "const": Fraction(2)},
    ]
    artificial_vars3 = 1
    result3 = two_phase_simplex(objective3, constraints3, artificial_vars3)
    print("\n--- Example 3 Result ---")
    print(f"Status: {result3[0]}")
    if result3[0] == "optimal":
        print(f"Optimal Solution: {result3[1]}")
        print(f"Optimal Value: {result3[2]}")



--- Phase 1 ---
Iteration 1 (Phase 1):
obj: {'x1': Fraction(0, 1), 'x2': Fraction(0, 1), 'const': Fraction(0, 1), 'a1': Fraction(1, 1)}
c0: {'x1': Fraction(1, 1), 'const': Fraction(1, 1), 'x2': Fraction(1, 1), 's1': Fraction(1, 1)}
c1: {'x1': Fraction(1, 1), 'const': Fraction(2, 1), 'x2': Fraction(1, 1), 's2': Fraction(-1, 1), 'a1': Fraction(1, 1)}
Entering variable: a1, Leaving variable: c1

Phase 1 Optimal Tableau:
c1: {'x1': Fraction(1, 1), 'const': Fraction(2, 1), 'x2': Fraction(1, 1), 's2': Fraction(-1, 1), 'a1': Fraction(1, 1)}
obj: {'const': Fraction(-2, 1), 's2': Fraction(1, 1), 'x2': Fraction(-1, 1), 'x1': Fraction(-1, 1), 'a1': Fraction(0, 1)}
c0: {'const': Fraction(1, 1), 's1': Fraction(1, 1), 's2': Fraction(0, 1), 'x2': Fraction(1, 1), 'x1': Fraction(1, 1), 'a1': Fraction(0, 1)}

--- Phase 2 ---
Iteration 1 (Phase 2):
obj: {'x1': Fraction(-1, 1), 'x2': Fraction(-1, 1), 'const': Fraction(0, 1)}
c1: {'x1': Fraction(1, 1), 'const': Fraction(2, 1), 'x2': Fraction(1, 1), 's2': 

In [88]:
# this is indeed unbounded
if True:
    # Example 4: Unbounded problem (Phase 2 might detect this)
    objective4 = {"x1": Fraction(-1), "x2": Fraction(-2), "const": Fraction(0)}
    constraints4 = [
        {"x1": Fraction(-1), "x2": Fraction(1), "type": "<=", "const": Fraction(1)},
        {"x1": Fraction(1), "type": ">=", "const": Fraction(2)},
    ]
    artificial_vars4 = 1
    result4 = two_phase_simplex(objective4, constraints4, artificial_vars4)
    print("\n--- Example 4 Result ---")
    print(f"Status: {result4[0]}")
    if result4[0] == "optimal":
        print(f"Optimal Solution: {result4[1]}")
        print(f"Optimal Value: {result4[2]}")


--- Phase 1 ---
Iteration 1 (Phase 1):
obj: {'x1': Fraction(0, 1), 'x2': Fraction(0, 1), 'const': Fraction(0, 1), 'a1': Fraction(1, 1)}
c0: {'x1': Fraction(-1, 1), 'const': Fraction(1, 1), 'x2': Fraction(1, 1), 's1': Fraction(1, 1)}
c1: {'x1': Fraction(1, 1), 'const': Fraction(2, 1), 's2': Fraction(-1, 1), 'a1': Fraction(1, 1)}
Entering variable: a1, Leaving variable: c1

Phase 1 Optimal Tableau:
c1: {'x1': Fraction(1, 1), 'const': Fraction(2, 1), 's2': Fraction(-1, 1), 'a1': Fraction(1, 1)}
obj: {'const': Fraction(-2, 1), 's2': Fraction(1, 1), 'x2': Fraction(0, 1), 'x1': Fraction(-1, 1), 'a1': Fraction(0, 1)}
c0: {'const': Fraction(1, 1), 's1': Fraction(1, 1), 's2': Fraction(0, 1), 'x2': Fraction(1, 1), 'x1': Fraction(-1, 1), 'a1': Fraction(0, 1)}

--- Phase 2 ---
Iteration 1 (Phase 2):
obj: {'x1': Fraction(-3, 1), 'const': Fraction(-2, 1)}
c1: {'x1': Fraction(1, 1), 'const': Fraction(2, 1), 's2': Fraction(-1, 1)}
c0: {'const': Fraction(1, 1), 's1': Fraction(1, 1), 's2': Fraction(0, 